In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
# Read data
Regression_data_month = pd.read_csv('./Regression_month_new.csv')

In [3]:
Regression_data_month.columns

Index(['Unnamed: 0', 'gvkey', 'eow_x', 'month', 'RET%', 'fyear', 'log_scope_1',
       'log_scope_2', 'Winsorised Growth Rate in Carbon Emissions Scope 1',
       'Winsorised Growth Rate in Carbon Emissions Scope 2',
       'Winsorised Carbon Intensity Scope 1',
       'Winsorised Carbon Intensity Scope 2', 'LOGSIZE', 'Winsorised BM',
       'Winsorised Leverage', 'Winsorised INVESTA', 'Winsorised ROE', 'LOGPPE',
       'Winsorised SALESGR', 'MOM_duplicate', 'beta', 'eow_y', 'mom_eow',
       'VOLAT', 'EPSGR', 'GICS_level_1'],
      dtype='object')

In [4]:
# Sort by 'gvkey', 'fyear' and 'month'
Regression_data_month = Regression_data_month.sort_values(by=['gvkey', 'fyear', 'month'])

columns_to_shift = ['log_scope_1', 'log_scope_2', 'Winsorised Growth Rate in Carbon Emissions Scope 1', 'Winsorised Growth Rate in Carbon Emissions Scope 2',
                    'Winsorised Carbon Intensity Scope 1', 'Winsorised Carbon Intensity Scope 2', 'LOGSIZE', 'Winsorised BM',
       'Winsorised Leverage', 'Winsorised INVESTA', 'Winsorised ROE', 'LOGPPE',
       'Winsorised SALESGR', 'MOM_duplicate', 'beta', 'VOLAT', 'EPSGR']

# Define the function to check and shift the variables
def shift_if_consecutive(group):
    group['year_month'] = pd.to_datetime(group['fyear'].astype(str) + '-' + group['month'].astype(str))
    group = group.sort_values(by='year_month')
    
    # Check fyear and month(include different years)
    group['consecutive'] = group['year_month'].diff().dt.days <= 31
    
    # Only shitf variables when time is continuous
    for col in columns_to_shift:
        group[f'{col}_t-1'] = group[col].shift(1)
        group[f'{col}_t-1'] = group.apply(lambda row: row[col] if not row['consecutive'] else row[f'{col}_t-1'], axis=1)
    
    return group.drop(columns=['consecutive', 'year_month'])

# Use function on every gvkey
Regression_data_month_shifted = Regression_data_month.groupby('gvkey').apply(shift_if_consecutive)

In [7]:
# Replace inf by using NaN
Regression_data_month_shifted = Regression_data_month_shifted.replace([np.inf, -np.inf], np.nan)

In [8]:
Regression_data_month_shifted.columns

Index(['Unnamed: 0', 'gvkey', 'eow_x', 'month', 'RET%', 'fyear', 'log_scope_1',
       'log_scope_2', 'Winsorised Growth Rate in Carbon Emissions Scope 1',
       'Winsorised Growth Rate in Carbon Emissions Scope 2',
       'Winsorised Carbon Intensity Scope 1',
       'Winsorised Carbon Intensity Scope 2', 'LOGSIZE', 'Winsorised BM',
       'Winsorised Leverage', 'Winsorised INVESTA', 'Winsorised ROE', 'LOGPPE',
       'Winsorised SALESGR', 'MOM_duplicate', 'beta', 'eow_y', 'mom_eow',
       'VOLAT', 'EPSGR', 'GICS_level_1', 'log_scope_1_t-1', 'log_scope_2_t-1',
       'Winsorised Growth Rate in Carbon Emissions Scope 1_t-1',
       'Winsorised Growth Rate in Carbon Emissions Scope 2_t-1',
       'Winsorised Carbon Intensity Scope 1_t-1',
       'Winsorised Carbon Intensity Scope 2_t-1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t

In [11]:
Regression_data_month_shifted.describe()

,Unnamed: 0,gvkey,month,RET%,fyear,log_scope_1,log_scope_2,Winsorised Growth Rate in Carbon Emissions Scope 1,Winsorised Growth Rate in Carbon Emissions Scope 2,Winsorised Carbon Intensity Scope 1,...,Winsorised BM_t-1,Winsorised Leverage_t-1,Winsorised INVESTA_t-1,Winsorised ROE_t-1,LOGPPE_t-1,Winsorised SALESGR_t-1,MOM_duplicate_t-1,beta_t-1,VOLAT_t-1,EPSGR_t-1
count,385419.000000,385419.000000,385419.000000,385419.000000,385419.000000,231039.000000,231064.000000,203232.000000,203232.000000,226999.000000,...,219103.000000,231464.000000,231534.000000,231879.000000,186682.000000,202646.000000,335404.000000,359830.000000,48952.000000,231334.000000
mean,192709.000000,54845.903910,6.554573,0.982939,2016.456693,10.196407,10.469592,0.063291,0.083948,1.263314,...,0.545848,0.270781,0.029671,7.726760,7.267478,0.090353,0.138830,1.107028,0.114921,0.025349
std,111261.026038,64289.213484,3.438300,11.314887,3.588310,3.124783,2.718884,0.370854,0.413537,4.693970,...,0.434137,0.202760,0.036302,41.568358,2.336629,1.797653,0.480396,0.627882,0.158213,1.784964
min,0.000000,1004.000000,1.000000,-46.052632,2010.000000,-1.645065,-0.079043,-0.766253,-0.676186,0.002459,...,-0.218942,0.000000,0.000000,-240.205070,-2.465104,-0.749541,-0.888750,-0.171023,0.023020,-7.513988
25%,96354.500000,10086.000000,4.000000,-4.822899,2013.000000,8.036567,8.764322,-0.060508,-0.062660,0.039278,...,0.255495,0.103979,0.003805,3.655294,5.711119,-0.008731,-0.120834,0.695566,0.060661,-0.551241
50%,192709.000000,23083.000000,7.000000,0.772200,2017.000000,10.186898,10.688325,0.000253,0.000000,0.130756,...,0.429435,0.249607,0.018633,9.840277,7.355641,0.033201,0.076577,1.056683,0.085914,-0.015222
75%,289063.500000,66366.000000,10.000000,6.319703,2019.000000,12.666913,12.793022,0.106248,0.132487,0.295027,...,0.762309,0.395576,0.039814,18.272460,9.074635,0.114613,0.300860,1.447234,0.126908,0.423447
max,385418.000000,328795.000000,12.000000,69.826889,2022.000000,18.804073,16.920606,2.102875,2.270040,34.648542,...,2.344154,0.904107,0.203190,186.515669,13.144907,225.869455,4.711340,2.832603,2.681848,12.620800


In [9]:
columns1 = ['RET%', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'Winsorised SALESGR', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'EPSGR', 'log_scope_1', 'log_scope_2', 'Winsorised Growth Rate in Carbon Emissions Scope 1', 'Winsorised Growth Rate in Carbon Emissions Scope 2', 'Winsorised Carbon Intensity Scope 1', 'Winsorised Carbon Intensity Scope 2']

Regression_data_1 = Regression_data_month_shifted.dropna(subset=columns1)

In [10]:
Regression_data_1.describe()

,Unnamed: 0,gvkey,month,RET%,fyear,log_scope_1,log_scope_2,Winsorised Growth Rate in Carbon Emissions Scope 1,Winsorised Growth Rate in Carbon Emissions Scope 2,Winsorised Carbon Intensity Scope 1,...,Winsorised BM_t-1,Winsorised Leverage_t-1,Winsorised INVESTA_t-1,Winsorised ROE_t-1,LOGPPE_t-1,Winsorised SALESGR_t-1,MOM_duplicate_t-1,beta_t-1,VOLAT_t-1,EPSGR_t-1
count,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,...,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000
mean,174744.212294,44435.211050,6.747382,-0.659493,2017.145457,11.253268,11.334847,0.065735,0.085762,1.365583,...,0.455822,0.296596,0.034513,9.868730,7.577355,0.071222,0.146147,1.148852,0.091512,0.016447
std,105855.145429,60178.609937,3.214942,9.819195,2.251616,2.718486,2.168211,0.375645,0.382755,4.938186,...,0.403352,0.194535,0.032812,43.424552,2.212868,0.482682,0.398813,0.610316,0.055841,1.769086
min,38.000000,1004.000000,1.000000,-45.584726,2011.000000,0.693147,2.544276,-0.766253,-0.676186,0.002459,...,-0.218942,0.000000,0.000000,-240.205070,-2.465104,-0.749541,-0.879796,-0.171023,0.023020,-7.456088
25%,80792.750000,6310.000000,4.000000,-6.092977,2015.000000,9.334521,9.754578,-0.059015,-0.056061,0.044903,...,0.210205,0.152195,0.014172,3.010917,6.053798,-0.011724,-0.078711,0.784278,0.053268,-0.605242
50%,170023.000000,12578.000000,7.000000,-0.354309,2017.000000,11.124618,11.386171,0.007182,0.001858,0.135847,...,0.342425,0.283332,0.023533,12.237104,7.646519,0.030993,0.121074,1.083779,0.077513,-0.063331
75%,259923.500000,61591.000000,9.000000,4.299363,2019.000000,13.474518,13.214941,0.114711,0.150427,0.317050,...,0.587599,0.415909,0.043512,20.104404,9.423434,0.111369,0.296220,1.465627,0.112489,0.418762
max,385411.000000,328795.000000,12.000000,69.616958,2020.000000,18.713283,16.862228,2.102875,2.270040,34.648542,...,2.344154,0.904107,0.203190,186.515669,13.144907,27.515772,4.392917,2.832603,1.086177,12.620800


In [9]:
# Create year dummies
year_dummies = pd.get_dummies(Regression_data_1['fyear'], prefix='fyear', drop_first=True)

# Add year dummies to dataframe
Regression_data_1 = pd.concat([Regression_data_1, year_dummies], axis=1)

In [10]:
# Create month dummies
month_dummies = pd.get_dummies(Regression_data_1['month'], prefix='month', drop_first=True)

# Add month dummies to dataframe
Regression_data_1 = pd.concat([Regression_data_1, month_dummies], axis=1)

In [11]:
# Create industry dummies
industry_dummies = pd.get_dummies(Regression_data_1['GICS_level_1'], prefix='industry', drop_first=True)

# Add industry dummies to dataframe
Regression_data_1 = pd.concat([Regression_data_1, industry_dummies], axis=1)

In [12]:
Regression_data_1.describe()

,Unnamed: 0,gvkey,month,RET%,fyear,log_scope_1,log_scope_2,Winsorised Growth Rate in Carbon Emissions Scope 1,Winsorised Growth Rate in Carbon Emissions Scope 2,Winsorised Carbon Intensity Scope 1,...,industry_Consumer Discretionary,industry_Consumer Staple,industry_Energy,industry_Financial,industry_Health Care,industry_Industrial,industry_Information Technology,industry_Material,industry_Real Estate,industry_Utilities
count,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,...,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000
mean,174744.212294,44435.211050,6.747382,-0.659493,2017.145457,11.253268,11.334847,0.065735,0.085762,1.365583,...,0.117505,0.048311,0.090469,0.071026,0.104543,0.307214,0.122087,0.054726,0.018722,0.035153
std,105855.145429,60178.609937,3.214942,9.819195,2.251616,2.718486,2.168211,0.375645,0.382755,4.938186,...,0.322031,0.214430,0.286861,0.256877,0.305974,0.461354,0.327397,0.227453,0.135547,0.184173
min,38.000000,1004.000000,1.000000,-45.584726,2011.000000,0.693147,2.544276,-0.766253,-0.676186,0.002459,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,80792.750000,6310.000000,4.000000,-6.092977,2015.000000,9.334521,9.754578,-0.059015,-0.056061,0.044903,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,170023.000000,12578.000000,7.000000,-0.354309,2017.000000,11.124618,11.386171,0.007182,0.001858,0.135847,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,259923.500000,61591.000000,9.000000,4.299363,2019.000000,13.474518,13.214941,0.114711,0.150427,0.317050,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,385411.000000,328795.000000,12.000000,69.616958,2020.000000,18.713283,16.862228,2.102875,2.270040,34.648542,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
Regression_data_1.to_csv('Regression_month_dummy.csv', index=True)

### log_scope_1 regression

In [81]:
"""
    Regression without fixed effects
"""

# Set up variables
Y_L1_1 = Regression_data_1['RET%']
X_L1_1 = Regression_data_1[['log_scope_1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1']]

# Add constant
X_L1_1 = sm.add_constant(X_L1_1)

# Construct OLS model
model_L1_1 = sm.OLS(Y_L1_1, X_L1_1)

# Fit the model
results_L1_1 = model_L1_1.fit()

# Output
print(results_L1_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     35.01
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           3.02e-81
Time:                        14:52:18   Log-Likelihood:                -56363.
No. Observations:               15276   AIC:                         1.128e+05
Df Residuals:                   15263   BIC:                         1.129e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [82]:
"""
    Regression with year fixed effects
"""

# Set up variables
Y_L1_2 = Regression_data_1['RET%']
X_L1_2 = Regression_data_1[['log_scope_1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020']]

# Add constant
X_L1_2 = sm.add_constant(X_L1_2)

# Construct OLS model
model_L1_2 = sm.OLS(Y_L1_2, X_L1_2)

# Fit the model
results_L1_2 = model_L1_2.fit()

# Output
print(results_L1_2.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     134.9
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:18   Log-Likelihood:                -55270.
No. Observations:               15276   AIC:                         1.106e+05
Df Residuals:                   15254   BIC:                         1.108e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [141]:
# Get coefficients, p-values and standard errors of independent variables
coefficients_L1_2 = results_L1_2.params
p_values_L1_2 = results_L1_2.pvalues
sd_L1_2 = results_L1_2.bse

# Print coefficients
print("Coefficients:\n", coefficients_L1_2)

Coefficients:
 const                      15.476074
log_scope_1                 0.070672
LOGSIZE_t-1                 0.334841
Winsorised BM_t-1          -2.448889
Winsorised Leverage_t-1    -1.605134
Winsorised INVESTA_t-1     -3.771450
Winsorised ROE_t-1          0.003881
LOGPPE_t-1                 -0.058160
Winsorised SALESGR_t-1     -0.197240
MOM_duplicate_t-1          -1.047930
beta_t-1                   -0.150063
VOLAT_t-1                  21.000819
EPSGR_t-1                   0.022662
fyear_2012                -16.211814
fyear_2013                -19.274095
fyear_2014                -16.593924
fyear_2015                -24.173565
fyear_2016                -21.172629
fyear_2017                -17.281547
fyear_2018                -24.266776
fyear_2019                -13.775879
fyear_2020                -20.369233
dtype: float64


In [142]:
# Print p-values
print("P-values:\n", p_values_L1_2)

P-values:
 const                      2.296739e-21
log_scope_1                1.479345e-01
LOGSIZE_t-1                3.766910e-05
Winsorised BM_t-1          4.779303e-30
Winsorised Leverage_t-1    1.108233e-04
Winsorised INVESTA_t-1     1.484269e-01
Winsorised ROE_t-1         2.566793e-02
LOGPPE_t-1                 5.186854e-01
Winsorised SALESGR_t-1     2.084097e-01
MOM_duplicate_t-1          3.585343e-07
beta_t-1                   2.519666e-01
VOLAT_t-1                  4.107815e-33
EPSGR_t-1                  5.890262e-01
fyear_2012                 1.210374e-24
fyear_2013                 1.561637e-34
fyear_2014                 9.981716e-28
fyear_2015                 3.494516e-55
fyear_2016                 7.312427e-43
fyear_2017                 5.030726e-30
fyear_2018                 2.480376e-57
fyear_2019                 1.675242e-19
fyear_2020                 6.060498e-41
dtype: float64


In [143]:
# Print standard errors
print("standard error:\n", sd_L1_2)

standard error:
 const                      1.628238
log_scope_1                0.048842
LOGSIZE_t-1                0.081222
Winsorised BM_t-1          0.214574
Winsorised Leverage_t-1    0.415128
Winsorised INVESTA_t-1     2.609669
Winsorised ROE_t-1         0.001739
LOGPPE_t-1                 0.090117
Winsorised SALESGR_t-1     0.156788
MOM_duplicate_t-1          0.205799
beta_t-1                   0.130987
VOLAT_t-1                  1.747669
EPSGR_t-1                  0.041946
fyear_2012                 1.579228
fyear_2013                 1.568740
fyear_2014                 1.517562
fyear_2015                 1.538734
fyear_2016                 1.537982
fyear_2017                 1.514823
fyear_2018                 1.514236
fyear_2019                 1.523052
fyear_2020                 1.515616
dtype: float64


In [85]:
"""
    Regression with year and month fixed effects
"""

# Set up variables
Y_L1_3 = Regression_data_1['RET%']
X_L1_3 = Regression_data_1[['log_scope_1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]

# Add constant
X_L1_3 = sm.add_constant(X_L1_3)

# Construct OLS model
model_L1_3 = sm.OLS(Y_L1_3, X_L1_3)

# Fit the model
results_L1_3 = model_L1_3.fit()

# Output
print(results_L1_3.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     222.1
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:19   Log-Likelihood:                -53648.
No. Observations:               15276   AIC:                         1.074e+05
Df Residuals:                   15243   BIC:                         1.076e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [145]:
# Get coefficients and p-values of independent variables
coefficients_L1_3 = results_L1_3.params
p_values_L1_3 = results_L1_3.pvalues
sd_L1_3 = results_L1_3.bse

# Print coefficients
print("Coefficients:\n", coefficients_L1_3)

Coefficients:
 const                      13.438190
log_scope_1                 0.087666
LOGSIZE_t-1                 0.439714
Winsorised BM_t-1          -1.822887
Winsorised Leverage_t-1    -1.668494
Winsorised INVESTA_t-1     -4.413203
Winsorised ROE_t-1          0.002449
LOGPPE_t-1                 -0.115073
Winsorised SALESGR_t-1     -0.145141
MOM_duplicate_t-1          -0.662494
beta_t-1                    0.301941
VOLAT_t-1                   8.953888
EPSGR_t-1                  -0.107393
fyear_2012                -15.538356
fyear_2013                -15.750980
fyear_2014                -15.199950
fyear_2015                -30.528064
fyear_2016                -22.107012
fyear_2017                -18.119569
fyear_2018                -23.419580
fyear_2019                -10.106426
fyear_2020                -22.210428
month_2                    -1.669204
month_3                    -1.729535
month_4                     2.960229
month_5                   -10.956232
month_6                

In [146]:
# Print p_values
print("P-values:\n", p_values_L1_3)

P-values:
 const                      8.543807e-17
log_scope_1                4.639702e-02
LOGSIZE_t-1                3.763747e-09
Winsorised BM_t-1          1.903419e-20
Winsorised Leverage_t-1    8.090614e-06
Winsorised INVESTA_t-1     6.357395e-02
Winsorised ROE_t-1         1.180184e-01
LOGPPE_t-1                 1.587084e-01
Winsorised SALESGR_t-1     3.036241e-01
MOM_duplicate_t-1          4.506913e-04
beta_t-1                   1.181223e-02
VOLAT_t-1                  3.073827e-08
EPSGR_t-1                  5.082718e-03
fyear_2012                 2.421326e-25
fyear_2013                 3.692377e-22
fyear_2014                 1.238133e-26
fyear_2015                 1.765264e-94
fyear_2016                 2.165048e-56
fyear_2017                 2.130302e-34
fyear_2018                 2.110048e-59
fyear_2019                 1.949821e-10
fyear_2020                 2.559844e-53
month_2                    1.617672e-04
month_3                    7.157716e-04
month_4                    1.

In [147]:
# Print standard errors
print("standard error:\n", sd_L1_3)

standard error:
 const                      1.612637
log_scope_1                0.044010
LOGSIZE_t-1                0.074557
Winsorised BM_t-1          0.196414
Winsorised Leverage_t-1    0.373741
Winsorised INVESTA_t-1     2.378712
Winsorised ROE_t-1         0.001567
LOGPPE_t-1                 0.081641
Winsorised SALESGR_t-1     0.141088
MOM_duplicate_t-1          0.188786
beta_t-1                   0.119913
VOLAT_t-1                  1.616209
EPSGR_t-1                  0.038325
fyear_2012                 1.491074
fyear_2013                 1.624759
fyear_2014                 1.420288
fyear_2015                 1.470073
fyear_2016                 1.391402
fyear_2017                 1.477822
fyear_2018                 1.434633
fyear_2019                 1.586671
fyear_2020                 1.439332
month_2                    0.442382
month_3                    0.511068
month_4                    0.606824
month_5                    0.909685
month_6                    0.752328
month_7    

In [88]:
"""
    Regression with year, month and industry fixed effects
"""

# Set up variables
Y_L1_4 = Regression_data_1['RET%']
X_L1_4 = Regression_data_1[['log_scope_1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'industry_Consumer Discretionary', 'industry_Consumer Staple','industry_Energy', 'industry_Financial', 'industry_Health Care',
        'industry_Industrial', 'industry_Information Technology','industry_Material', 'industry_Real Estate', 'industry_Utilities']]

# Add constant
X_L1_4 = sm.add_constant(X_L1_4)

# Construct OLS model
model_L1_4 = sm.OLS(Y_L1_4, X_L1_4)

# Fit the model
results_L1_4 = model_L1_4.fit()

# Output
print(results_L1_4.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     170.5
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:19   Log-Likelihood:                -53628.
No. Observations:               15276   AIC:                         1.073e+05
Df Residuals:                   15233   BIC:                         1.077e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [148]:
# Get coefficients and p-values of independent variables
coefficients_L1_4 = results_L1_4.params
p_values_L1_4 = results_L1_4.pvalues
sd_L1_4 = results_L1_4.bse

# Print coefficients
print("Coefficients:\n", coefficients_L1_4)

Coefficients:
 const                              13.726935
log_scope_1                         0.004063
LOGSIZE_t-1                         0.587395
Winsorised BM_t-1                  -1.641407
Winsorised Leverage_t-1            -1.598097
Winsorised INVESTA_t-1             -4.282128
Winsorised ROE_t-1                  0.001727
LOGPPE_t-1                         -0.184225
Winsorised SALESGR_t-1             -0.087336
MOM_duplicate_t-1                  -0.667467
beta_t-1                            0.245981
VOLAT_t-1                          10.268899
EPSGR_t-1                          -0.103874
fyear_2012                        -15.684582
fyear_2013                        -15.770204
fyear_2014                        -15.320032
fyear_2015                        -30.611842
fyear_2016                        -22.302481
fyear_2017                        -18.250761
fyear_2018                        -23.605060
fyear_2019                        -10.165546
fyear_2020                        -22.34

In [149]:
# Print p-values
print("P-values:\n", p_values_L1_4)

P-values:
 const                              1.201328e-16
log_scope_1                        9.412280e-01
LOGSIZE_t-1                        1.126180e-12
Winsorised BM_t-1                  3.281853e-13
Winsorised Leverage_t-1            4.539575e-05
Winsorised INVESTA_t-1             7.950172e-02
Winsorised ROE_t-1                 2.746211e-01
LOGPPE_t-1                         2.952785e-02
Winsorised SALESGR_t-1             5.376098e-01
MOM_duplicate_t-1                  4.190881e-04
beta_t-1                           4.759099e-02
VOLAT_t-1                          6.530118e-10
EPSGR_t-1                          6.732150e-03
fyear_2012                         8.366900e-26
fyear_2013                         3.780144e-22
fyear_2014                         4.837722e-27
fyear_2015                         4.644623e-95
fyear_2016                         2.340440e-57
fyear_2017                         6.826964e-35
fyear_2018                         2.600002e-60
fyear_2019                   

In [150]:
# Print standard errors
print("standard error:\n", sd_L1_4)

standard error:
 const                              1.655353
log_scope_1                        0.055109
LOGSIZE_t-1                        0.082497
Winsorised BM_t-1                  0.225197
Winsorised Leverage_t-1            0.391755
Winsorised INVESTA_t-1             2.441765
Winsorised ROE_t-1                 0.001581
LOGPPE_t-1                         0.084640
Winsorised SALESGR_t-1             0.141676
MOM_duplicate_t-1                  0.189162
beta_t-1                           0.124160
VOLAT_t-1                          1.661353
EPSGR_t-1                          0.038327
fyear_2012                         1.490614
fyear_2013                         1.627145
fyear_2014                         1.419914
fyear_2015                         1.469440
fyear_2016                         1.391338
fyear_2017                         1.477341
fyear_2018                         1.434608
fyear_2019                         1.588912
fyear_2020                         1.439098
month_2        

### log_scope_2 regression

In [91]:
"""
    Regression without fixed effects
"""

# Set up variables
Y_L2_1 = Regression_data_1['RET%']
X_L2_1 = Regression_data_1[['log_scope_2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1']]

# Add constant
X_L2_1 = sm.add_constant(X_L2_1)

# Construct OLS model
model_L2_1 = sm.OLS(Y_L2_1, X_L2_1)

# Fit the model
results_L2_1 = model_L2_1.fit()

# Output
print(results_L2_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     34.76
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           1.28e-80
Time:                        14:52:19   Log-Likelihood:                -56365.
No. Observations:               15276   AIC:                         1.128e+05
Df Residuals:                   15263   BIC:                         1.129e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [151]:
"""
    Regression with year fixed effects
"""

# Set up variables
Y_L2_2 = Regression_data_1['RET%']
X_L2_2 = Regression_data_1[['log_scope_2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020']]

# Add constant
X_L2_2 = sm.add_constant(X_L2_2)

# Construct OLS model
model_L2_2 = sm.OLS(Y_L2_2, X_L2_2)

# Fit the model
results_L2_2 = model_L2_2.fit()

# Output
print(results_L2_2.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     134.8
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:35:26   Log-Likelihood:                -55271.
No. Observations:               15276   AIC:                         1.106e+05
Df Residuals:                   15254   BIC:                         1.108e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [152]:
# Get coefficients and p-values of independent variables
coefficients_L2_2 = results_L2_2.params
p_values_L2_2 = results_L2_2.pvalues
sd_L2_2 = results_L2_2.bse

# Print coefficients
print("Coefficients:\n", coefficients_L2_2)

Coefficients:
 const                      15.985172
log_scope_2                -0.026860
LOGSIZE_t-1                 0.321013
Winsorised BM_t-1          -2.441111
Winsorised Leverage_t-1    -1.608651
Winsorised INVESTA_t-1     -3.878502
Winsorised ROE_t-1          0.003991
LOGPPE_t-1                  0.042208
Winsorised SALESGR_t-1     -0.190902
MOM_duplicate_t-1          -1.046513
beta_t-1                   -0.153830
VOLAT_t-1                  20.886391
EPSGR_t-1                   0.021937
fyear_2012                -16.223553
fyear_2013                -19.289136
fyear_2014                -16.606842
fyear_2015                -24.201562
fyear_2016                -21.197143
fyear_2017                -17.305960
fyear_2018                -24.295212
fyear_2019                -13.824649
fyear_2020                -20.427368
dtype: float64


In [153]:
# Print p-values
print("P-values:\n", p_values_L2_2)

P-values:
 const                      1.570180e-22
log_scope_2                6.855855e-01
LOGSIZE_t-1                7.963961e-05
Winsorised BM_t-1          8.849372e-30
Winsorised Leverage_t-1    1.072477e-04
Winsorised INVESTA_t-1     1.416715e-01
Winsorised ROE_t-1         2.220061e-02
LOGPPE_t-1                 6.135986e-01
Winsorised SALESGR_t-1     2.232586e-01
MOM_duplicate_t-1          3.829553e-07
beta_t-1                   2.409251e-01
VOLAT_t-1                  7.737931e-33
EPSGR_t-1                  6.009901e-01
fyear_2012                 1.133307e-24
fyear_2013                 1.405344e-34
fyear_2014                 9.214196e-28
fyear_2015                 2.720738e-55
fyear_2016                 6.079667e-43
fyear_2017                 4.245934e-30
fyear_2018                 1.879168e-57
fyear_2019                 1.274574e-19
fyear_2020                 3.970663e-41
dtype: float64


In [154]:
# Print standard errors
print("standard error:\n", sd_L2_2)

standard error:
 const                      1.634176
log_scope_2                0.066344
LOGSIZE_t-1                0.081340
Winsorised BM_t-1          0.214912
Winsorised Leverage_t-1    0.415179
Winsorised INVESTA_t-1     2.639027
Winsorised ROE_t-1         0.001745
LOGPPE_t-1                 0.083588
Winsorised SALESGR_t-1     0.156739
MOM_duplicate_t-1          0.206028
beta_t-1                   0.131173
VOLAT_t-1                  1.745839
EPSGR_t-1                  0.041946
fyear_2012                 1.579388
fyear_2013                 1.568865
fyear_2014                 1.517728
fyear_2015                 1.538938
fyear_2016                 1.538254
fyear_2017                 1.514988
fyear_2018                 1.514353
fyear_2019                 1.523395
fyear_2020                 1.516373
dtype: float64


In [155]:
"""
    Regression with year and month fixed effects
"""

# Set up variables
Y_L2_3 = Regression_data_1['RET%']
X_L2_3 = Regression_data_1[['log_scope_2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]

# Add constant
X_L2_3 = sm.add_constant(X_L2_3)

# Construct OLS model
model_L2_3 = sm.OLS(Y_L2_3, X_L2_3)

# Fit the model
results_L2_3 = model_L2_3.fit()

# Output
print(results_L2_3.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     222.0
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:44:48   Log-Likelihood:                -53649.
No. Observations:               15276   AIC:                         1.074e+05
Df Residuals:                   15243   BIC:                         1.076e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [156]:
# Get coefficients and p-values of independent variables
coefficients_L2_3 = results_L2_3.params
p_values_L2_3 = results_L2_3.pvalues
sd_L2_3 = results_L2_3.bse

# Print coefficients
print("Coefficients:\n", coefficients_L2_3)

Coefficients:
 const                      13.651623
log_scope_2                 0.060756
LOGSIZE_t-1                 0.402254
Winsorised BM_t-1          -1.831326
Winsorised Leverage_t-1    -1.682020
Winsorised INVESTA_t-1     -4.002388
Winsorised ROE_t-1          0.002380
LOGPPE_t-1                 -0.054514
Winsorised SALESGR_t-1     -0.133309
MOM_duplicate_t-1          -0.645137
beta_t-1                    0.284570
VOLAT_t-1                   8.769750
EPSGR_t-1                  -0.108661
fyear_2012                -15.538481
fyear_2013                -15.747775
fyear_2014                -15.186923
fyear_2015                -30.535021
fyear_2016                -22.104090
fyear_2017                -18.135028
fyear_2018                -23.436505
fyear_2019                -10.109869
fyear_2020                -22.204550
month_2                    -1.689803
month_3                    -1.754129
month_4                     2.966927
month_5                   -10.981165
month_6                

In [157]:
# Print p-values
print("P-values:\n", p_values_L2_3)

P-values:
 const                      3.717634e-17
log_scope_2                3.098266e-01
LOGSIZE_t-1                6.665383e-08
Winsorised BM_t-1          1.399457e-20
Winsorised Leverage_t-1    6.854540e-06
Winsorised INVESTA_t-1     9.566583e-02
Winsorised ROE_t-1         1.301949e-01
LOGPPE_t-1                 4.705413e-01
Winsorised SALESGR_t-1     3.446009e-01
MOM_duplicate_t-1          6.447996e-04
beta_t-1                   1.780425e-02
VOLAT_t-1                  5.684839e-08
EPSGR_t-1                  4.588220e-03
fyear_2012                 2.446808e-25
fyear_2013                 3.852407e-22
fyear_2014                 1.391321e-26
fyear_2015                 1.693813e-94
fyear_2016                 2.341627e-56
fyear_2017                 1.908107e-34
fyear_2018                 1.779933e-59
fyear_2019                 1.962815e-10
fyear_2020                 3.061377e-53
month_2                    1.337843e-04
month_3                    5.980057e-04
month_4                    1.

In [158]:
# Print standard errors
print("standard error:\n", sd_L2_3)

standard error:
 const                      1.619137
log_scope_2                0.059821
LOGSIZE_t-1                0.074455
Winsorised BM_t-1          0.196626
Winsorised Leverage_t-1    0.373805
Winsorised INVESTA_t-1     2.401900
Winsorised ROE_t-1         0.001572
LOGPPE_t-1                 0.075544
Winsorised SALESGR_t-1     0.141046
MOM_duplicate_t-1          0.189038
beta_t-1                   0.120076
VOLAT_t-1                  1.614712
EPSGR_t-1                  0.038328
fyear_2012                 1.491230
fyear_2013                 1.625159
fyear_2014                 1.420516
fyear_2015                 1.470263
fyear_2016                 1.391656
fyear_2017                 1.477997
fyear_2018                 1.434742
fyear_2019                 1.587466
fyear_2020                 1.440046
month_2                    0.442315
month_3                    0.510926
month_4                    0.607025
month_5                    0.910137
month_6                    0.752389
month_7    

In [159]:
"""
    Regression with year, month and industry fixed effects
"""

# Set up variables
Y_L2_4 = Regression_data_1['RET%']
X_L2_4 = Regression_data_1[['log_scope_2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'industry_Consumer Discretionary', 'industry_Consumer Staple','industry_Energy', 'industry_Financial', 'industry_Health Care',
        'industry_Industrial', 'industry_Information Technology','industry_Material', 'industry_Real Estate', 'industry_Utilities']]

# Add constant
X_L2_4 = sm.add_constant(X_L2_4)

# Construct OLS model
model_L2_4 = sm.OLS(Y_L2_4, X_L2_4)

# Fit the model
results_L2_4 = model_L2_4.fit()

# 打印结果
print(results_L2_4.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     170.5
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:57:05   Log-Likelihood:                -53628.
No. Observations:               15276   AIC:                         1.073e+05
Df Residuals:                   15233   BIC:                         1.077e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [160]:
# Get coefficients and p-values of independent variables
coefficients_L2_4 = results_L2_4.params
p_values_L2_4 = results_L2_4.pvalues
sd_L2_4 = results_L2_4.bse

# Print coefficients
print("Coefficients:\n", coefficients_L2_4)

Coefficients:
 const                              13.914937
log_scope_2                        -0.046824
LOGSIZE_t-1                         0.602581
Winsorised BM_t-1                  -1.613075
Winsorised Leverage_t-1            -1.589789
Winsorised INVESTA_t-1             -4.487846
Winsorised ROE_t-1                  0.001790
LOGPPE_t-1                         -0.153254
Winsorised SALESGR_t-1             -0.087267
MOM_duplicate_t-1                  -0.671929
beta_t-1                            0.247303
VOLAT_t-1                          10.285011
EPSGR_t-1                          -0.103451
fyear_2012                        -15.687325
fyear_2013                        -15.792465
fyear_2014                        -15.329714
fyear_2015                        -30.623357
fyear_2016                        -22.317156
fyear_2017                        -18.266932
fyear_2018                        -23.614518
fyear_2019                        -10.203377
fyear_2020                        -22.37

In [161]:
# Print p-values
print("P-values:\n", p_values_L2_4)

P-values:
 const                              7.370147e-17
log_scope_2                        4.987613e-01
LOGSIZE_t-1                        1.322937e-12
Winsorised BM_t-1                  1.028579e-12
Winsorised Leverage_t-1            4.963958e-05
Winsorised INVESTA_t-1             6.812657e-02
Winsorised ROE_t-1                 2.578367e-01
LOGPPE_t-1                         7.559987e-02
Winsorised SALESGR_t-1             5.378587e-01
MOM_duplicate_t-1                  3.860718e-04
beta_t-1                           4.642807e-02
VOLAT_t-1                          6.110599e-10
EPSGR_t-1                          6.964318e-03
fyear_2012                         8.124822e-26
fyear_2013                         3.361668e-22
fyear_2014                         4.491120e-27
fyear_2015                         3.863950e-95
fyear_2016                         1.940830e-57
fyear_2017                         5.866786e-35
fyear_2018                         2.201089e-60
fyear_2019                   

In [162]:
# Print standard errors
print("standard error:\n", sd_L2_4)

standard error:
 const                              1.666338
log_scope_2                        0.069219
LOGSIZE_t-1                        0.084896
Winsorised BM_t-1                  0.226152
Winsorised Leverage_t-1            0.391720
Winsorised INVESTA_t-1             2.460039
Winsorised ROE_t-1                 0.001582
LOGPPE_t-1                         0.086247
Winsorised SALESGR_t-1             0.141652
MOM_duplicate_t-1                  0.189264
beta_t-1                           0.124171
VOLAT_t-1                          1.661139
EPSGR_t-1                          0.038331
fyear_2012                         1.490481
fyear_2013                         1.627419
fyear_2014                         1.419906
fyear_2015                         1.469353
fyear_2016                         1.391227
fyear_2017                         1.477178
fyear_2018                         1.434289
fyear_2019                         1.589594
fyear_2020                         1.439403
month_2        

### Growth rate scope 1 regression

In [101]:
"""
    Regression without fixed effects
"""

# Set up variables
Y_G1_1 = Regression_data_1['RET%']
X_G1_1 = Regression_data_1[['Winsorised Growth Rate in Carbon Emissions Scope 1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1']]

# Add constant
X_G1_1 = sm.add_constant(X_G1_1)

# Construct OLS model
model_G1_1 = sm.OLS(Y_G1_1, X_G1_1)

# Fit the model
results_G1_1 = model_G1_1.fit()

# Output
print(results_G1_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     34.74
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           1.45e-80
Time:                        14:52:19   Log-Likelihood:                -56365.
No. Observations:               15276   AIC:                         1.128e+05
Df Residuals:                   15263   BIC:                         1.129e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [163]:
"""
    Regression with year fixed effects
"""

# Set up variables
Y_G1_2 = Regression_data_1['RET%']
X_G1_2 = Regression_data_1[['Winsorised Growth Rate in Carbon Emissions Scope 1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020']]

# Add constant
X_G1_2 = sm.add_constant(X_G1_2)

# Construct OLS model
model_G1_2 = sm.OLS(Y_G1_2, X_G1_2)

# Fit the model
results_G1_2 = model_G1_2.fit()

# Output
print(results_G1_2.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     134.7
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:12:47   Log-Likelihood:                -55271.
No. Observations:               15276   AIC:                         1.106e+05
Df Residuals:                   15254   BIC:                         1.108e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [164]:
# Get coefficients and p-values of independent variables
coefficients_G1_2 = results_G1_2.params
p_values_G1_2 = results_G1_2.pvalues
sd_G1_2 = results_G1_2.bse

# Print coefficients
print("Coefficients:\n", coefficients_G1_2)

Coefficients:
 const                                                 15.865017
Winsorised Growth Rate in Carbon Emissions Scope 1     0.055652
LOGSIZE_t-1                                            0.314688
Winsorised BM_t-1                                     -2.447824
Winsorised Leverage_t-1                               -1.614947
Winsorised INVESTA_t-1                                -3.763309
Winsorised ROE_t-1                                     0.003929
LOGPPE_t-1                                             0.024516
Winsorised SALESGR_t-1                                -0.205771
MOM_duplicate_t-1                                     -1.045171
beta_t-1                                              -0.157308
VOLAT_t-1                                             20.873984
EPSGR_t-1                                              0.021667
fyear_2012                                           -16.214910
fyear_2013                                           -19.283794
fyear_2014               

In [165]:
# Print p-value
print("P-values:\n", p_values_G1_2)

P-values:
 const                                                 6.122117e-23
Winsorised Growth Rate in Carbon Emissions Scope 1    7.929272e-01
LOGSIZE_t-1                                           8.619114e-05
Winsorised BM_t-1                                     5.444977e-30
Winsorised Leverage_t-1                               1.015942e-04
Winsorised INVESTA_t-1                                1.501298e-01
Winsorised ROE_t-1                                    2.389340e-02
LOGPPE_t-1                                            7.267804e-01
Winsorised SALESGR_t-1                                2.212429e-01
MOM_duplicate_t-1                                     3.973474e-07
beta_t-1                                              2.294895e-01
VOLAT_t-1                                             8.292178e-33
EPSGR_t-1                                             6.054977e-01
fyear_2012                                            1.197036e-24
fyear_2013                                         

In [166]:
# Print standard errors
print("standard error:\n", sd_G1_2)

standard error:
 const                                                 1.606218
Winsorised Growth Rate in Carbon Emissions Scope 1    0.211994
LOGSIZE_t-1                                           0.080123
Winsorised BM_t-1                                     0.214696
Winsorised Leverage_t-1                               0.415391
Winsorised INVESTA_t-1                                2.614960
Winsorised ROE_t-1                                    0.001739
LOGPPE_t-1                                            0.070162
Winsorised SALESGR_t-1                                0.168212
MOM_duplicate_t-1                                     0.206049
beta_t-1                                              0.130902
VOLAT_t-1                                             1.745648
EPSGR_t-1                                             0.041948
fyear_2012                                            1.579364
fyear_2013                                            1.568829
fyear_2014                            

In [105]:
"""
    Regression with year and month fixed effects
"""

# Set up variables
Y_G1_3 = Regression_data_1['RET%']
X_G1_3 = Regression_data_1[['Winsorised Growth Rate in Carbon Emissions Scope 1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]

# Add constant
X_G1_3 = sm.add_constant(X_G1_3)

# Construct OLS model
model_G1_3 = sm.OLS(Y_G1_3, X_G1_3)

# Fit the model
results_G1_3 = model_G1_3.fit()

# Output
print(results_G1_3.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     222.0
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:19   Log-Likelihood:                -53649.
No. Observations:               15276   AIC:                         1.074e+05
Df Residuals:                   15243   BIC:                         1.076e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [167]:
# Get coefficients and p-values of independent variables
coefficients_G1_3 = results_G1_3.params
p_values_G1_3 = results_G1_3.pvalues
sd_G1_3 = results_G1_3.bse

# Print coefficients
print("Coefficients:\n", coefficients_G1_3)

Coefficients:
 const                                                 13.930742
Winsorised Growth Rate in Carbon Emissions Scope 1     0.187836
LOGSIZE_t-1                                            0.413062
Winsorised BM_t-1                                     -1.826109
Winsorised Leverage_t-1                               -1.688284
Winsorised INVESTA_t-1                                -4.483895
Winsorised ROE_t-1                                     0.002502
LOGPPE_t-1                                            -0.011143
Winsorised SALESGR_t-1                                -0.189875
MOM_duplicate_t-1                                     -0.665176
beta_t-1                                               0.293214
VOLAT_t-1                                              8.789922
EPSGR_t-1                                             -0.108850
fyear_2012                                           -15.528945
fyear_2013                                           -15.775547
fyear_2014               

In [168]:
# Print p-values
print("P-values:\n", p_values_G1_3)

P-values:
 const                                                 2.374733e-18
Winsorised Growth Rate in Carbon Emissions Scope 1    3.278029e-01
LOGSIZE_t-1                                           1.937662e-08
Winsorised BM_t-1                                     1.672886e-20
Winsorised Leverage_t-1                               6.394464e-06
Winsorised INVESTA_t-1                                5.992359e-02
Winsorised ROE_t-1                                    1.102924e-01
LOGPPE_t-1                                            8.610315e-01
Winsorised SALESGR_t-1                                2.099708e-01
MOM_duplicate_t-1                                     4.348307e-04
beta_t-1                                              1.442082e-02
VOLAT_t-1                                             5.279188e-08
EPSGR_t-1                                             4.520623e-03
fyear_2012                                            2.631733e-25
fyear_2013                                         

In [169]:
# Print standard errors
print("standard error:\n", sd_G1_3)

standard error:
 const                                                 1.592266
Winsorised Growth Rate in Carbon Emissions Scope 1    0.191948
LOGSIZE_t-1                                           0.073492
Winsorised BM_t-1                                     0.196468
Winsorised Leverage_t-1                               0.373970
Winsorised INVESTA_t-1                                2.383155
Winsorised ROE_t-1                                    0.001567
LOGPPE_t-1                                            0.063651
Winsorised SALESGR_t-1                                0.151452
MOM_duplicate_t-1                                     0.189037
beta_t-1                                              0.119832
VOLAT_t-1                                             1.614496
EPSGR_t-1                                             0.038331
fyear_2012                                            1.491314
fyear_2013                                            1.624860
fyear_2014                            

In [108]:
"""
    Regression with year, month and industry fixed effects
"""

# Set up variables
Y_G1_4 = Regression_data_1['RET%']
X_G1_4 = Regression_data_1[['Winsorised Growth Rate in Carbon Emissions Scope 1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'industry_Consumer Discretionary', 'industry_Consumer Staple','industry_Energy', 'industry_Financial', 'industry_Health Care',
        'industry_Industrial', 'industry_Information Technology','industry_Material', 'industry_Real Estate', 'industry_Utilities']]

# Add constant
X_G1_4 = sm.add_constant(X_G1_4)

# Construct OLS model
model_G1_4 = sm.OLS(Y_G1_4, X_G1_4)

# Fit the model
results_G1_4 = model_G1_4.fit()

# Output
print(results_G1_4.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     170.5
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:19   Log-Likelihood:                -53627.
No. Observations:               15276   AIC:                         1.073e+05
Df Residuals:                   15233   BIC:                         1.077e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [172]:
# Get coefficients and p-values of independent variables
coefficients_G1_4 = results_G1_4.params
p_values_G1_4 = results_G1_4.pvalues
sd_G1_4 = results_G1_4.bse

# Print coefficients
print("Coefficients:\n", coefficients_G1_4)

Coefficients:
 const                                                 13.706598
Winsorised Growth Rate in Carbon Emissions Scope 1     0.244430
LOGSIZE_t-1                                            0.586900
Winsorised BM_t-1                                     -1.641075
Winsorised Leverage_t-1                               -1.609509
Winsorised INVESTA_t-1                                -4.427004
Winsorised ROE_t-1                                     0.001698
LOGPPE_t-1                                            -0.179342
Winsorised SALESGR_t-1                                -0.157084
MOM_duplicate_t-1                                     -0.679968
beta_t-1                                               0.245681
VOLAT_t-1                                             10.284753
EPSGR_t-1                                             -0.104567
fyear_2012                                           -15.665054
fyear_2013                                           -15.762848
fyear_2014               

In [173]:
# Print p-values
print("P-values:\n", p_values_G1_4)

P-values:
 const                                                 9.069759e-17
Winsorised Growth Rate in Carbon Emissions Scope 1    2.054115e-01
LOGSIZE_t-1                                           9.218723e-13
Winsorised BM_t-1                                     1.885474e-13
Winsorised Leverage_t-1                               3.988840e-05
Winsorised INVESTA_t-1                                7.011012e-02
Winsorised ROE_t-1                                    2.825298e-01
LOGPPE_t-1                                            1.762150e-02
Winsorised SALESGR_t-1                                3.015121e-01
MOM_duplicate_t-1                                     3.315023e-04
beta_t-1                                              4.784837e-02
VOLAT_t-1                                             6.083238e-10
EPSGR_t-1                                             6.373481e-03
fyear_2012                                            9.533746e-26
fyear_2013                                         

In [174]:
# Print standard errors
print("standard error:\n", sd_G1_4)

standard error:
 const                                                 1.646249
Winsorised Growth Rate in Carbon Emissions Scope 1    0.193021
LOGSIZE_t-1                                           0.082109
Winsorised BM_t-1                                     0.222871
Winsorised Leverage_t-1                               0.391672
Winsorised INVESTA_t-1                                2.444068
Winsorised ROE_t-1                                    0.001580
LOGPPE_t-1                                            0.075553
Winsorised SALESGR_t-1                                0.152031
MOM_duplicate_t-1                                     0.189397
beta_t-1                                              0.124152
VOLAT_t-1                                             1.660907
EPSGR_t-1                                             0.038327
fyear_2012                                            1.490514
fyear_2013                                            1.627054
fyear_2014                            

### Growth rate scope 2 regression

In [111]:
"""
    Regression without fixed effects
"""

# Set up variables
Y_G2_1 = Regression_data_1['RET%']
X_G2_1 = Regression_data_1[['Winsorised Growth Rate in Carbon Emissions Scope 2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1']]

# Add constant
X_G2_1 = sm.add_constant(X_G2_1)

# Construct OLS model
model_G2_1 = sm.OLS(Y_G2_1, X_G2_1)

# Fit the model
results_G2_1 = model_G2_1.fit()

# Output
print(results_G2_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     35.12
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           1.64e-81
Time:                        14:52:19   Log-Likelihood:                -56363.
No. Observations:               15276   AIC:                         1.128e+05
Df Residuals:                   15263   BIC:                         1.129e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [112]:
"""
    Regression with year fixed effects
"""

# Set up variables
Y_G2_2 = Regression_data_1['RET%']
X_G2_2 = Regression_data_1[['Winsorised Growth Rate in Carbon Emissions Scope 2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020']]

# Add constant
X_G2_2 = sm.add_constant(X_G2_2)

# Construct OLS model
model_G2_2 = sm.OLS(Y_G2_2, X_G2_2)

# Fit the model
results_G2_2 = model_G2_2.fit()

# Output
print(results_G2_2.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     134.8
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:19   Log-Likelihood:                -55271.
No. Observations:               15276   AIC:                         1.106e+05
Df Residuals:                   15254   BIC:                         1.108e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [175]:
# Get coefficients and p-values of independent variables
coefficients_G2_2 = results_G2_2.params
p_values_G2_2 = results_G2_2.pvalues
sd_G2_2 = results_G2_2.bse

# Print coefficients
print("Coefficients:\n", coefficients_G2_2)

Coefficients:
 const                                                 15.868622
Winsorised Growth Rate in Carbon Emissions Scope 2    -0.169330
LOGSIZE_t-1                                            0.317866
Winsorised BM_t-1                                     -2.437268
Winsorised Leverage_t-1                               -1.594193
Winsorised INVESTA_t-1                                -3.566838
Winsorised ROE_t-1                                     0.003924
LOGPPE_t-1                                             0.020175
Winsorised SALESGR_t-1                                -0.140049
MOM_duplicate_t-1                                     -1.034731
beta_t-1                                              -0.160342
VOLAT_t-1                                             20.909072
EPSGR_t-1                                              0.022538
fyear_2012                                           -16.226482
fyear_2013                                           -19.291231
fyear_2014               

In [176]:
# Print p-values
print("P-values:\n", p_values_G2_2)

P-values:
 const                                                 5.976771e-23
Winsorised Growth Rate in Carbon Emissions Scope 2    4.274703e-01
LOGSIZE_t-1                                           7.358318e-05
Winsorised BM_t-1                                     1.046946e-29
Winsorised Leverage_t-1                               1.259740e-04
Winsorised INVESTA_t-1                                1.728552e-01
Winsorised ROE_t-1                                    2.404904e-02
LOGPPE_t-1                                            7.739719e-01
Winsorised SALESGR_t-1                                4.065957e-01
MOM_duplicate_t-1                                     5.145682e-07
beta_t-1                                              2.208262e-01
VOLAT_t-1                                             6.779196e-33
EPSGR_t-1                                             5.911353e-01
fyear_2012                                            1.108064e-24
fyear_2013                                         

In [177]:
# Print standard errors
print("standard error:\n", sd_G2_2)

standard error:
 const                                                 1.606189
Winsorised Growth Rate in Carbon Emissions Scope 2    0.213383
LOGSIZE_t-1                                           0.080158
Winsorised BM_t-1                                     0.214854
Winsorised Leverage_t-1                               0.415676
Winsorised INVESTA_t-1                                2.616613
Winsorised ROE_t-1                                    0.001739
LOGPPE_t-1                                            0.070251
Winsorised SALESGR_t-1                                0.168750
MOM_duplicate_t-1                                     0.206000
beta_t-1                                              0.130958
VOLAT_t-1                                             1.746118
EPSGR_t-1                                             0.041955
fyear_2012                                            1.579337
fyear_2013                                            1.568822
fyear_2014                            

In [115]:
"""
    Regression with year and month fixed effects
"""

# Set up variables
Y_G2_3 = Regression_data_1['RET%']
X_G2_3 = Regression_data_1[['Winsorised Growth Rate in Carbon Emissions Scope 2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]

# Add constant
X_G2_3 = sm.add_constant(X_G2_3)

# Construct OLS model
model_G2_3 = sm.OLS(Y_G2_3, X_G2_3)

# Fit the model
results_G2_3 = model_G2_3.fit()

# Output
print(results_G2_3.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     222.0
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:20   Log-Likelihood:                -53649.
No. Observations:               15276   AIC:                         1.074e+05
Df Residuals:                   15243   BIC:                         1.076e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [178]:
# Get coefficients and p-values of independent variables
coefficients_G2_3 = results_G2_3.params
p_values_G2_3 = results_G2_3.pvalues
sd_G2_3 = results_G2_3.bse

# Print coefficients
print("Coefficients:\n", coefficients_G2_3)

Coefficients:
 const                                                 13.949661
Winsorised Growth Rate in Carbon Emissions Scope 2     0.194099
LOGSIZE_t-1                                            0.411691
Winsorised BM_t-1                                     -1.832591
Winsorised Leverage_t-1                               -1.696329
Winsorised INVESTA_t-1                                -4.523633
Winsorised ROE_t-1                                     0.002521
LOGPPE_t-1                                            -0.008685
Winsorised SALESGR_t-1                                -0.192941
MOM_duplicate_t-1                                     -0.663895
beta_t-1                                               0.295219
VOLAT_t-1                                              8.760244
EPSGR_t-1                                             -0.109266
fyear_2012                                           -15.534401
fyear_2013                                           -15.748192
fyear_2014               

In [179]:
# Print p-values
print("P-values:\n", p_values_G2_3)

P-values:
 const                                                 2.124616e-18
Winsorised Growth Rate in Carbon Emissions Scope 2    3.138662e-01
LOGSIZE_t-1                                           2.193478e-08
Winsorised BM_t-1                                     1.355314e-20
Winsorised Leverage_t-1                               5.888225e-06
Winsorised INVESTA_t-1                                5.794667e-02
Winsorised ROE_t-1                                    1.075994e-01
LOGPPE_t-1                                            8.916756e-01
Winsorised SALESGR_t-1                                2.043241e-01
MOM_duplicate_t-1                                     4.436804e-04
beta_t-1                                              1.378878e-02
VOLAT_t-1                                             5.896818e-08
EPSGR_t-1                                             4.377614e-03
fyear_2012                                            2.522425e-25
fyear_2013                                         

In [181]:
# Print standard errors
print("standard error:\n", sd_G2_3)

standard error:
 const                                                 1.592133
Winsorised Growth Rate in Carbon Emissions Scope 2    0.192716
LOGSIZE_t-1                                           0.073530
Winsorised BM_t-1                                     0.196690
Winsorised Leverage_t-1                               0.374303
Winsorised INVESTA_t-1                                2.385576
Winsorised ROE_t-1                                    0.001567
LOGPPE_t-1                                            0.063772
Winsorised SALESGR_t-1                                0.151996
MOM_duplicate_t-1                                     0.188961
beta_t-1                                              0.119861
VOLAT_t-1                                             1.614909
EPSGR_t-1                                             0.038339
fyear_2012                                            1.491255
fyear_2013                                            1.625157
fyear_2014                            

In [182]:
"""
    Regression with year, month and industry fixed effects
"""

# Set up variables
Y_G2_4 = Regression_data_1['RET%']
X_G2_4 = Regression_data_1[['Winsorised Growth Rate in Carbon Emissions Scope 2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'industry_Consumer Discretionary', 'industry_Consumer Staple','industry_Energy', 'industry_Financial', 'industry_Health Care',
        'industry_Industrial', 'industry_Information Technology','industry_Material', 'industry_Real Estate', 'industry_Utilities']]

# Add constant
X_G2_4 = sm.add_constant(X_G2_4)

# Construct OLS model
model_G2_4 = sm.OLS(Y_G2_4, X_G2_4)

# Fit the model
results_G2_4 = model_G2_4.fit()

# Output
print(results_G2_4.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     170.5
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        17:07:39   Log-Likelihood:                -53628.
No. Observations:               15276   AIC:                         1.073e+05
Df Residuals:                   15233   BIC:                         1.077e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [183]:
# Get coefficients and p-values of independent variables
coefficients_G2_4 = results_G2_4.params
p_values_G2_4 = results_G2_4.pvalues
sd_G2_4 = results_G2_4.bse

# Print coefficients
print("Coefficients:\n", coefficients_G2_4)

Coefficients:
 const                                                 13.730023
Winsorised Growth Rate in Carbon Emissions Scope 2     0.234279
LOGSIZE_t-1                                            0.584652
Winsorised BM_t-1                                     -1.646040
Winsorised Leverage_t-1                               -1.615839
Winsorised INVESTA_t-1                                -4.452229
Winsorised ROE_t-1                                     0.001720
LOGPPE_t-1                                            -0.175690
Winsorised SALESGR_t-1                                -0.155170
MOM_duplicate_t-1                                     -0.676894
beta_t-1                                               0.246861
VOLAT_t-1                                             10.230175
EPSGR_t-1                                             -0.104999
fyear_2012                                           -15.671306
fyear_2013                                           -15.727182
fyear_2014               

In [184]:
# Print p-values
print("P-values:\n", p_values_G2_4)

P-values:
 const                                                 7.984577e-17
Winsorised Growth Rate in Carbon Emissions Scope 2    2.260432e-01
LOGSIZE_t-1                                           1.154141e-12
Winsorised BM_t-1                                     1.624239e-13
Winsorised Leverage_t-1                               3.750409e-05
Winsorised INVESTA_t-1                                6.868099e-02
Winsorised ROE_t-1                                    2.761034e-01
LOGPPE_t-1                                            2.027897e-02
Winsorised SALESGR_t-1                                3.085563e-01
MOM_duplicate_t-1                                     3.502572e-04
beta_t-1                                              4.679195e-02
VOLAT_t-1                                             7.522409e-10
EPSGR_t-1                                             6.168897e-03
fyear_2012                                            9.097833e-26
fyear_2013                                         

In [185]:
# Print standard errors
print("standard error:\n", sd_G2_4)

standard error:
 const                                                 1.646067
Winsorised Growth Rate in Carbon Emissions Scope 2    0.193513
LOGSIZE_t-1                                           0.082151
Winsorised BM_t-1                                     0.222942
Winsorised Leverage_t-1                               0.391855
Winsorised INVESTA_t-1                                2.445418
Winsorised ROE_t-1                                    0.001580
LOGPPE_t-1                                            0.075683
Winsorised SALESGR_t-1                                0.152383
MOM_duplicate_t-1                                     0.189299
beta_t-1                                              0.124154
VOLAT_t-1                                             1.661115
EPSGR_t-1                                             0.038334
fyear_2012                                            1.490479
fyear_2013                                            1.627447
fyear_2014                            

### Carbon intensity scope 1 regression

In [121]:
"""
    Regression without fixed effects
"""

# Set up variables
Y_I1_1 = Regression_data_1['RET%']
X_I1_1 = Regression_data_1[['Winsorised Carbon Intensity Scope 1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1']]

# Add constant
X_I1_1 = sm.add_constant(X_I1_1)

# Construct OLS model
model_I1_1 = sm.OLS(Y_I1_1, X_I1_1)

# Fit the model
results_I1_1 = model_I1_1.fit()

# Output
print(results_I1_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     34.80
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           1.00e-80
Time:                        14:52:20   Log-Likelihood:                -56365.
No. Observations:               15276   AIC:                         1.128e+05
Df Residuals:                   15263   BIC:                         1.129e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [122]:
"""
    Regression with year fixed effects
"""

# Set up variables
Y_I1_2 = Regression_data_1['RET%']
X_I1_2 = Regression_data_1[['Winsorised Carbon Intensity Scope 1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020']]

# Add constant
X_I1_2 = sm.add_constant(X_I1_2)

# Construct OLS model
model_I1_2 = sm.OLS(Y_I1_2, X_I1_2)

# Fit the model
results_I1_2 = model_I1_2.fit()

# Output
print(results_I1_2.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     134.9
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:20   Log-Likelihood:                -55269.
No. Observations:               15276   AIC:                         1.106e+05
Df Residuals:                   15254   BIC:                         1.108e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [186]:
# Get coefficients and p-values of independent variables
coefficients_I1_2 = results_I1_2.params
p_values_I1_2 = results_I1_2.pvalues
sd_I1_2 = results_I1_2.bse

# Print coefficients
print("Coefficients:\n", coefficients_I1_2)

Coefficients:
 const                                  15.803404
Winsorised Carbon Intensity Scope 1     0.027130
LOGSIZE_t-1                             0.320996
Winsorised BM_t-1                      -2.443819
Winsorised Leverage_t-1                -1.601638
Winsorised INVESTA_t-1                 -3.714142
Winsorised ROE_t-1                      0.004059
LOGPPE_t-1                              0.020592
Winsorised SALESGR_t-1                 -0.188557
MOM_duplicate_t-1                      -1.044685
beta_t-1                               -0.155173
VOLAT_t-1                              20.962326
EPSGR_t-1                               0.017520
fyear_2012                            -16.233073
fyear_2013                            -19.302206
fyear_2014                            -16.617512
fyear_2015                            -24.207779
fyear_2016                            -21.203247
fyear_2017                            -17.317781
fyear_2018                            -24.311705
fyear

In [187]:
# Print p-values
print("P-values:\n", p_values_I1_2)

P-values:
 const                                  9.028623e-23
Winsorised Carbon Intensity Scope 1    6.760457e-02
LOGSIZE_t-1                            6.226810e-05
Winsorised BM_t-1                      6.198443e-30
Winsorised Leverage_t-1                1.146847e-04
Winsorised INVESTA_t-1                 1.546360e-01
Winsorised ROE_t-1                     1.969038e-02
LOGPPE_t-1                             7.690188e-01
Winsorised SALESGR_t-1                 2.288721e-01
MOM_duplicate_t-1                      3.875077e-07
beta_t-1                               2.358387e-01
VOLAT_t-1                              4.699183e-33
EPSGR_t-1                              6.766283e-01
fyear_2012                             1.050355e-24
fyear_2013                             1.247680e-34
fyear_2014                             8.392786e-28
fyear_2015                             2.436485e-55
fyear_2016                             5.522525e-43
fyear_2017                             3.806258e-30
f

In [188]:
# Print standard errors
print("standard error:\n", sd_I1_2)

standard error:
 const                                  1.606366
Winsorised Carbon Intensity Scope 1    0.014843
LOGSIZE_t-1                            0.080145
Winsorised BM_t-1                      0.214559
Winsorised Leverage_t-1                0.415123
Winsorised INVESTA_t-1                 2.609315
Winsorised ROE_t-1                     0.001740
LOGPPE_t-1                             0.070120
Winsorised SALESGR_t-1                 0.156697
MOM_duplicate_t-1                      0.205759
beta_t-1                               0.130893
VOLAT_t-1                              1.746097
EPSGR_t-1                              0.042006
fyear_2012                             1.579181
fyear_2013                             1.568689
fyear_2014                             1.517521
fyear_2015                             1.538638
fyear_2016                             1.537913
fyear_2017                             1.514753
fyear_2018                             1.514154
fyear_2019             

In [125]:
"""
    Regression with year, month and industry fixed effects
"""

# Set up variables
Y_I1_3 = Regression_data_1['RET%']
X_I1_3 = Regression_data_1[['Winsorised Carbon Intensity Scope 1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]

# Add constant
X_I1_3 = sm.add_constant(X_I1_3)

# Construct OLS model
model_I1_3 = sm.OLS(Y_I1_3, X_I1_3)

# Fit the model
results_I1_3 = model_I1_3.fit()

# Output
print(results_I1_3.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     221.9
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:20   Log-Likelihood:                -53649.
No. Observations:               15276   AIC:                         1.074e+05
Df Residuals:                   15243   BIC:                         1.076e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [189]:
# Get coefficients and p-values of independent variables
coefficients_I1_3 = results_I1_3.params
p_values_I1_3 = results_I1_3.pvalues
sd_I1_3 = results_I1_3.bse

# Print coefficients
print("Coefficients:\n", coefficients_I1_3)

Coefficients:
 const                                  13.944639
Winsorised Carbon Intensity Scope 1     0.005191
LOGSIZE_t-1                             0.415358
Winsorised BM_t-1                      -1.822737
Winsorised Leverage_t-1                -1.674026
Winsorised INVESTA_t-1                 -4.335685
Winsorised ROE_t-1                      0.002539
LOGPPE_t-1                             -0.013772
Winsorised SALESGR_t-1                 -0.135586
MOM_duplicate_t-1                      -0.656190
beta_t-1                                0.292816
VOLAT_t-1                               8.817292
EPSGR_t-1                              -0.109013
fyear_2012                            -15.547213
fyear_2013                            -15.787620
fyear_2014                            -15.204536
fyear_2015                            -30.551829
fyear_2016                            -22.127584
fyear_2017                            -18.158721
fyear_2018                            -23.449971
fyear

In [190]:
# Print p-values
print("P-values:\n", p_values_I1_3)

P-values:
 const                                  2.197525e-18
Winsorised Carbon Intensity Scope 1    6.980704e-01
LOGSIZE_t-1                            1.634526e-08
Winsorised BM_t-1                      1.938867e-20
Winsorised Leverage_t-1                7.576556e-06
Winsorised INVESTA_t-1                 6.837476e-02
Winsorised ROE_t-1                     1.054321e-01
LOGPPE_t-1                             8.286812e-01
Winsorised SALESGR_t-1                 3.363677e-01
MOM_duplicate_t-1                      5.107354e-04
beta_t-1                               1.455857e-02
VOLAT_t-1                              4.871605e-08
EPSGR_t-1                              4.504816e-03
fyear_2012                             2.307219e-25
fyear_2013                             3.008893e-22
fyear_2014                             1.213504e-26
fyear_2015                             1.322223e-94
fyear_2016                             1.768327e-56
fyear_2017                             1.554030e-34
f

In [191]:
# Print standard errors
print("standard error:\n", sd_I1_3)

standard error:
 const                                  1.592255
Winsorised Carbon Intensity Scope 1    0.013382
LOGSIZE_t-1                            0.073517
Winsorised BM_t-1                      0.196440
Winsorised Leverage_t-1                0.373804
Winsorised INVESTA_t-1                 2.378770
Winsorised ROE_t-1                     0.001568
LOGPPE_t-1                             0.063641
Winsorised SALESGR_t-1                 0.141030
MOM_duplicate_t-1                      0.188788
beta_t-1                               0.119837
VOLAT_t-1                              1.615277
EPSGR_t-1                              0.038373
fyear_2012                             1.491262
fyear_2013                             1.625017
fyear_2014                             1.420467
fyear_2015                             1.470207
fyear_2016                             1.391568
fyear_2017                             1.477907
fyear_2018                             1.434774
fyear_2019             

In [128]:
"""
    Regression with year, month and industry fixed effects
"""

# Set up variables
Y_I1_4 = Regression_data_1['RET%']
X_I1_4 = Regression_data_1[['Winsorised Carbon Intensity Scope 1', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'industry_Consumer Discretionary', 'industry_Consumer Staple','industry_Energy', 'industry_Financial', 'industry_Health Care',
        'industry_Industrial', 'industry_Information Technology','industry_Material', 'industry_Real Estate', 'industry_Utilities']]

# Add constant
X_I1_4 = sm.add_constant(X_I1_4)

# Construct OLS model
model_I1_4 = sm.OLS(Y_I1_4, X_I1_4)

# Fit the model
results_I1_4 = model_I1_4.fit()

# Output
print(results_I1_4.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     170.5
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:20   Log-Likelihood:                -53628.
No. Observations:               15276   AIC:                         1.073e+05
Df Residuals:                   15233   BIC:                         1.077e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [192]:
# Get coefficients and p-values of independent variables
coefficients_I1_4 = results_I1_4.params
p_values_I1_4= results_I1_4.pvalues
sd_I1_4 = results_I1_4.bse

# Print coefficients
print("Coefficients:\n", coefficients_I1_4)

Coefficients:
 const                                  13.728519
Winsorised Carbon Intensity Scope 1     0.010072
LOGSIZE_t-1                             0.590528
Winsorised BM_t-1                      -1.637797
Winsorised Leverage_t-1                -1.592101
Winsorised INVESTA_t-1                 -4.256794
Winsorised ROE_t-1                      0.001768
LOGPPE_t-1                             -0.183588
Winsorised SALESGR_t-1                 -0.085991
MOM_duplicate_t-1                      -0.668178
beta_t-1                                0.245346
VOLAT_t-1                              10.303076
EPSGR_t-1                              -0.105295
fyear_2012                            -15.689468
fyear_2013                            -15.780978
fyear_2014                            -15.322334
fyear_2015                            -30.611643
fyear_2016                            -22.310183
fyear_2017                            -18.256823
fyear_2018                            -23.615045
fyear

In [193]:
# Print p-values
print("P-values:\n", p_values_I1_4)

P-values:
 const                                  8.079930e-17
Winsorised Carbon Intensity Scope 1    4.539835e-01
LOGSIZE_t-1                            6.982769e-13
Winsorised BM_t-1                      2.108223e-13
Winsorised Leverage_t-1                4.819597e-05
Winsorised INVESTA_t-1                 8.130433e-02
Winsorised ROE_t-1                     2.632015e-01
LOGPPE_t-1                             1.516796e-02
Winsorised SALESGR_t-1                 5.438444e-01
MOM_duplicate_t-1                      4.125720e-04
beta_t-1                               4.816587e-02
VOLAT_t-1                              5.770670e-10
EPSGR_t-1                              6.071974e-03
fyear_2012                             8.003231e-26
fyear_2013                             3.546080e-22
fyear_2014                             4.725787e-27
fyear_2015                             4.442158e-95
fyear_2016                             2.058510e-57
fyear_2017                             6.290554e-35
f

In [194]:
# Print standard errors
print("standard error:\n", sd_I1_4)

standard error:
 const                                  1.646165
Winsorised Carbon Intensity Scope 1    0.013451
LOGSIZE_t-1                            0.082178
Winsorised BM_t-1                      0.222879
Winsorised Leverage_t-1                0.391625
Winsorised INVESTA_t-1                 2.441826
Winsorised ROE_t-1                     0.001580
LOGPPE_t-1                             0.075594
Winsorised SALESGR_t-1                 0.141660
MOM_duplicate_t-1                      0.189142
beta_t-1                               0.124159
VOLAT_t-1                              1.661631
EPSGR_t-1                              0.038369
fyear_2012                             1.490482
fyear_2013                             1.627154
fyear_2014                             1.419842
fyear_2015                             1.469275
fyear_2016                             1.391115
fyear_2017                             1.477037
fyear_2018                             1.434281
fyear_2019             

### Carbon intensity scope 2 regression

In [131]:
"""
    Regression without fixed effects
"""

# Set up variables
Y_I2_1 = Regression_data_1['RET%']
X_I2_1 = Regression_data_1[['Winsorised Carbon Intensity Scope 2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1']]

# Add constant
X_I2_1 = sm.add_constant(X_I2_1)

# Construct OLS model
model_I2_1 = sm.OLS(Y_I2_1, X_I2_1)

# Fit the model
results_I2_1 = model_I2_1.fit()

# Output
print(results_I2_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     35.20
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           1.02e-81
Time:                        14:52:20   Log-Likelihood:                -56362.
No. Observations:               15276   AIC:                         1.128e+05
Df Residuals:                   15263   BIC:                         1.128e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [132]:
"""
    Regression with year fixed effects
"""

# Set up variables
Y_I2_2 = Regression_data_1['RET%']
X_I2_2 = Regression_data_1[['Winsorised Carbon Intensity Scope 2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020']]

# Add constant
X_I2_2 = sm.add_constant(X_I2_2)

# Construct OLS model
model_I2_2 = sm.OLS(Y_I2_2, X_I2_2)

# Fit the model
results_I2_2 = model_I2_2.fit()

# Output
print(results_I2_2.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     136.3
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:20   Log-Likelihood:                -55257.
No. Observations:               15276   AIC:                         1.106e+05
Df Residuals:                   15254   BIC:                         1.107e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [195]:
# Get coefficients and p-values of independent variables
coefficients_I2_2 = results_I2_2.params
p_values_I2_2 = results_I2_2.pvalues
sd_I2_2 = results_I2_2.bse

# Print coefficients
print("Coefficients:\n", coefficients_I2_2)

Coefficients:
 const                                  15.640582
Winsorised Carbon Intensity Scope 2     0.852021
LOGSIZE_t-1                             0.324216
Winsorised BM_t-1                      -2.431638
Winsorised Leverage_t-1                -1.582748
Winsorised INVESTA_t-1                 -2.951391
Winsorised ROE_t-1                      0.003887
LOGPPE_t-1                              0.008281
Winsorised SALESGR_t-1                 -0.183713
MOM_duplicate_t-1                      -1.075567
beta_t-1                               -0.159784
VOLAT_t-1                              20.947645
EPSGR_t-1                               0.027336
fyear_2012                            -16.252625
fyear_2013                            -19.326119
fyear_2014                            -16.660490
fyear_2015                            -24.208141
fyear_2016                            -21.319749
fyear_2017                            -17.341069
fyear_2018                            -24.414106
fyear

In [196]:
# Print p-values
print("P-values:\n", p_values_I2_2)

P-values:
 const                                  2.288488e-22
Winsorised Carbon Intensity Scope 2    9.788692e-08
LOGSIZE_t-1                            5.127677e-05
Winsorised BM_t-1                      1.071219e-29
Winsorised Leverage_t-1                1.362573e-04
Winsorised INVESTA_t-1                 2.583651e-01
Winsorised ROE_t-1                     2.531018e-02
LOGPPE_t-1                             9.059717e-01
Winsorised SALESGR_t-1                 2.406733e-01
MOM_duplicate_t-1                      1.723915e-07
beta_t-1                               2.218154e-01
VOLAT_t-1                              4.397549e-33
EPSGR_t-1                              5.143404e-01
fyear_2012                             8.466759e-25
fyear_2013                             9.128199e-35
fyear_2014                             5.576955e-28
fyear_2015                             1.978936e-55
fyear_2016                             1.705922e-43
fyear_2017                             2.863008e-30
f

In [198]:
# Print standard errors
print("standard error:\n", sd_I2_2)

standard error:
 const                                  1.605256
Winsorised Carbon Intensity Scope 2    0.159759
LOGSIZE_t-1                            0.080036
Winsorised BM_t-1                      0.214396
Winsorised Leverage_t-1                0.414784
Winsorised INVESTA_t-1                 2.611144
Winsorised ROE_t-1                     0.001738
LOGPPE_t-1                             0.070101
Winsorised SALESGR_t-1                 0.156571
MOM_duplicate_t-1                      0.205680
beta_t-1                               0.130781
VOLAT_t-1                              1.744070
EPSGR_t-1                              0.041919
fyear_2012                             1.577874
fyear_2013                             1.567390
fyear_2014                             1.516288
fyear_2015                             1.537354
fyear_2016                             1.536825
fyear_2017                             1.513493
fyear_2018                             1.513040
fyear_2019             

In [135]:
"""
    Regression with year and month fixed effects
"""

# Set up variables
Y_I2_3 = Regression_data_1['RET%']
X_I2_3 = Regression_data_1[['Winsorised Carbon Intensity Scope 2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]

# Add constant
X_I2_3 = sm.add_constant(X_I2_3)

# Construct OLS model
model_I2_3 = sm.OLS(Y_I2_3, X_I2_3)

# Fit the model
results_I2_3 = model_I2_3.fit()

# Output
print(results_I2_3.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     222.0
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:20   Log-Likelihood:                -53649.
No. Observations:               15276   AIC:                         1.074e+05
Df Residuals:                   15243   BIC:                         1.076e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [200]:
# Get coefficients and p-values of independent variables
coefficients_I2_3 = results_I2_3.params
p_values_I2_3 = results_I2_3.pvalues
sd_I2_3 = results_I2_3.bse

# Print coefficients
print("Coefficients:\n", coefficients_I2_3)

Coefficients:
 const                                  13.869898
Winsorised Carbon Intensity Scope 2     0.177009
LOGSIZE_t-1                             0.415900
Winsorised BM_t-1                      -1.821335
Winsorised Leverage_t-1                -1.669302
Winsorised INVESTA_t-1                 -4.185732
Winsorised ROE_t-1                      0.002508
LOGPPE_t-1                             -0.016049
Winsorised SALESGR_t-1                 -0.134559
MOM_duplicate_t-1                      -0.663960
beta_t-1                                0.291554
VOLAT_t-1                               8.831433
EPSGR_t-1                              -0.106390
fyear_2012                            -15.536404
fyear_2013                            -15.771895
fyear_2014                            -15.179500
fyear_2015                            -30.504473
fyear_2016                            -22.148598
fyear_2017                            -18.125409
fyear_2018                            -23.456589
fyear

In [201]:
# Print p-values
print("P-values:\n", p_values_I2_3)

P-values:
 const                                  3.523984e-18
Winsorised Carbon Intensity Scope 2    2.226939e-01
LOGSIZE_t-1                            1.545099e-08
Winsorised BM_t-1                      2.065372e-20
Winsorised Leverage_t-1                8.032594e-06
Winsorised INVESTA_t-1                 7.890159e-02
Winsorised ROE_t-1                     1.094072e-01
LOGPPE_t-1                             8.009669e-01
Winsorised SALESGR_t-1                 3.400257e-01
MOM_duplicate_t-1                      4.412074e-04
beta_t-1                               1.498180e-02
VOLAT_t-1                              4.584947e-08
EPSGR_t-1                              5.549727e-03
fyear_2012                             2.479088e-25
fyear_2013                             3.272519e-22
fyear_2014                             1.473858e-26
fyear_2015                             2.893911e-94
fyear_2016                             1.409273e-56
fyear_2017                             2.075921e-34
f

In [202]:
# Print standard errors
print("standard error:\n", sd_I2_3)

standard error:
 const                                  1.593487
Winsorised Carbon Intensity Scope 2    0.145155
LOGSIZE_t-1                            0.073487
Winsorised BM_t-1                      0.196432
Winsorised Leverage_t-1                0.373793
Winsorised INVESTA_t-1                 2.382029
Winsorised ROE_t-1                     0.001567
LOGPPE_t-1                             0.063662
Winsorised SALESGR_t-1                 0.141026
MOM_duplicate_t-1                      0.188899
beta_t-1                               0.119829
VOLAT_t-1                              1.614678
EPSGR_t-1                              0.038357
fyear_2012                             1.491210
fyear_2013                             1.624839
fyear_2014                             1.420538
fyear_2015                             1.470667
fyear_2016                             1.391625
fyear_2017                             1.478043
fyear_2018                             1.434709
fyear_2019             

In [138]:
"""
    Regression with year, month and industry fixed effects
"""

# Set up variables
Y_I2_4 = Regression_data_1['RET%']
X_I2_4 = Regression_data_1[['Winsorised Carbon Intensity Scope 2', 'LOGSIZE_t-1',
       'Winsorised BM_t-1', 'Winsorised Leverage_t-1',
       'Winsorised INVESTA_t-1', 'Winsorised ROE_t-1', 'LOGPPE_t-1',
       'Winsorised SALESGR_t-1', 'MOM_duplicate_t-1', 'beta_t-1',
       'VOLAT_t-1', 'EPSGR_t-1', 'fyear_2012', 'fyear_2013', 'fyear_2014', 'fyear_2015', 'fyear_2016',
       'fyear_2017', 'fyear_2018', 'fyear_2019', 'fyear_2020', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'industry_Consumer Discretionary', 'industry_Consumer Staple','industry_Energy', 'industry_Financial', 'industry_Health Care',
        'industry_Industrial', 'industry_Information Technology','industry_Material', 'industry_Real Estate', 'industry_Utilities']]

# Add constant
X_I2_4 = sm.add_constant(X_I2_4)

# Construct OLS model
model_I2_4 = sm.OLS(Y_I2_4, X_I2_4)

# Fit the model
results_I2_4 = model_I2_4.fit()

# Output
print(results_I2_4.summary())

                            OLS Regression Results                            
Dep. Variable:                   RET%   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     170.5
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:52:20   Log-Likelihood:                -53628.
No. Observations:               15276   AIC:                         1.073e+05
Df Residuals:                   15233   BIC:                         1.077e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [204]:
# Get coefficients and p-values of independent variables
coefficients_I2_4 = results_I2_4.params
p_values_I2_4 = results_I2_4.pvalues
sd_I2_4 = results_I2_4.bse

# Print coefficients
print("Coefficients:\n", coefficients_I2_4)

Coefficients:
 const                                  13.672813
Winsorised Carbon Intensity Scope 2     0.158992
LOGSIZE_t-1                             0.588238
Winsorised BM_t-1                      -1.641968
Winsorised Leverage_t-1                -1.595860
Winsorised INVESTA_t-1                 -4.169980
Winsorised ROE_t-1                      0.001732
LOGPPE_t-1                             -0.182857
Winsorised SALESGR_t-1                 -0.086025
MOM_duplicate_t-1                      -0.675167
beta_t-1                                0.245955
VOLAT_t-1                              10.288149
EPSGR_t-1                              -0.102250
fyear_2012                            -15.677599
fyear_2013                            -15.769011
fyear_2014                            -15.301518
fyear_2015                            -30.572301
fyear_2016                            -22.326504
fyear_2017                            -18.227571
fyear_2018                            -23.616221
fyear

In [205]:
# Print p-values
print("P-values:\n", p_values_I2_4)

P-values:
 const                                  1.121704e-16
Winsorised Carbon Intensity Scope 2    2.739343e-01
LOGSIZE_t-1                            8.174073e-13
Winsorised BM_t-1                      1.835895e-13
Winsorised Leverage_t-1                4.611688e-05
Winsorised INVESTA_t-1                 8.795445e-02
Winsorised ROE_t-1                     2.729346e-01
LOGPPE_t-1                             1.551463e-02
Winsorised SALESGR_t-1                 5.436611e-01
MOM_duplicate_t-1                      3.619424e-04
beta_t-1                               4.760273e-02
VOLAT_t-1                              6.016202e-10
EPSGR_t-1                              7.684639e-03
fyear_2012                             8.691506e-26
fyear_2013                             3.789405e-22
fyear_2014                             5.572116e-27
fyear_2015                             8.693257e-95
fyear_2016                             1.739952e-57
fyear_2017                             8.144267e-35
f

In [206]:
# Print standard errors
print("standard error:\n", sd_I2_4)

standard error:
 const                                  1.647201
Winsorised Carbon Intensity Scope 2    0.145320
LOGSIZE_t-1                            0.082106
Winsorised BM_t-1                      0.222884
Winsorised Leverage_t-1                0.391559
Winsorised INVESTA_t-1                 2.443733
Winsorised ROE_t-1                     0.001579
LOGPPE_t-1                             0.075548
Winsorised SALESGR_t-1                 0.141652
MOM_duplicate_t-1                      0.189272
beta_t-1                               0.124153
VOLAT_t-1                              1.660986
EPSGR_t-1                              0.038354
fyear_2012                             1.490462
fyear_2013                             1.627064
fyear_2014                             1.419922
fyear_2015                             1.469721
fyear_2016                             1.391211
fyear_2017                             1.477178
fyear_2018                             1.434238
fyear_2019             